<a href="https://colab.research.google.com/github/Hritikrai55/retail-analytics-for-store-optimization/blob/main/Retail_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project: Integrated Retail Analytics for Store Optimization and Demand Forecasting**
Dataset:

1. Feature Dataset
2. Sales Dataset
3. Stores Dataset

## **Project Objective:**
  - To utilize machine learning and data analysis techniques to optimize store performance, forecast demand, and enhance customer experience through segmentation and personalized marketing strategies.

## **Project Components:**

### **Anomaly Detection in Sales Data:**
* Identify unusual sales patterns across stores and departments.
* Investigate potential causes (e.g., holidays, markdowns, economic indicators).
* Implement anomaly handling strategies to clean the data for further analysis.

### **Time-Based Anomaly Detection:**
* Analyze sales trends over time.
* Detect seasonal variations and holiday effects on sales.
* Use time-series analysis for understanding store and department performance over time.

### **Data Preprocessing and Feature Engineering:**
* Handle missing values, especially in the MarkDown data.
* Create new features that could influence sales (e.g., store size/type, regional factors).

### **Customer Segmentation Analysis:**
* Segment stores or departments based on sales patterns, markdowns, and regional features.
* Analyze segment-specific trends and characteristics.

### **Market Basket Analysis:**
* Although individual customer transaction data is not available, infer potential product associations within departments using sales data.
* Develop cross-selling strategies based on these inferences.

### **Demand Forecasting:**
* Build models to forecast weekly sales for each store and department.
* Incorporate factors like CPI, unemployment rate, fuel prices, and store/dept attributes.
* Explore short-term and long-term forecasting models.

### **Impact of External Factors:**
* Examine how external factors (economic indicators, regional climate) influence sales.
* Incorporate these insights into the demand forecasting models.

### **Personalization Strategies:**
* Develop personalized marketing strategies based on the markdowns and store segments.
* Propose inventory management strategies tailored to store and department needs.

### **Segmentation Quality Evaluation:**
* Evaluate the effectiveness of the customer segmentation.
* Use metrics to assess the quality of segments in terms of homogeneity and separation.

### **Real-World Application and Strategy Formulation:**
* Formulate a comprehensive strategy for inventory management, marketing, and store optimization based on the insights gathered.
* Discuss potential real-world challenges in implementing these strategies.

## **About Dataset**

**Content**
* You are provided with historical sales data for 45 stores located in different regions - each store contains a number of departments. The company also runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks.
* **Markdown:** *Markdown is a deliberate reduction in the original selling price of a product. Retailers use markdowns to clear out old or slow-moving inventory, make room for new products, and recoup some of the costs associated with carrying unsold merchandise.*

Within the Excel Sheet, there are 3 Tabs – Stores, Features and Sales

**Stores**
* Anonymized information about the 45 stores, indicating the type and size of store

**Features**
* Contains additional data related to the store, department, and regional activity for the given dates.
* Store - the store number
* Date - the week
* Temperature - average temperature in the region
* Fuel_Price - cost of fuel in the region
* MarkDown1-5 - anonymized data related to promotional markdowns. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA
* CPI - the consumer price index
* Unemployment - the unemployment rate
* IsHoliday - whether the week is a special holiday week

**Sales**
* Historical sales data, which covers to 2010-02-05 to 2012-11-01. Within this tab you will find the following fields:
* Store - the store number
* Dept - the department number
* Date - the week
* Weekly_Sales -  sales for the given department in the given store
* IsHoliday - whether the week is a special holiday week

### **Task:**
- Predict the department-wise sales for each store for the following year
- Model the effects of markdowns on holiday weeks
- Provide recommended actions based on the insights drawn, with prioritization placed on largest business impact

#**Importing Libraries and Data Ingestion**

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
feature_df=pd.read_csv("Features data set.csv")
sales_df=pd.read_csv("sales data-set.csv")
store_df=pd.read_csv("stores data-set.csv")

In [3]:
feature_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [4]:
sales_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [5]:
store_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [6]:
feature_df.shape,sales_df.shape,store_df.shape

((8190, 12), (421570, 5), (45, 3))

In [7]:
# Display the info for each DataFrame in a systematic way
print("Feature DataFrame Information:")
print(feature_df.nunique())
print("\n" + "-"*50 + "\n")

print("Sales DataFrame Information:")
print(sales_df.nunique())
print("\n" + "-"*50 + "\n")

print("Store DataFrame Information:")
print(store_df.nunique())

Feature DataFrame Information:
Store             45
Date             182
Temperature     4178
Fuel_Price      1011
MarkDown1       4023
MarkDown2       2715
MarkDown3       2885
MarkDown4       3405
MarkDown5       4045
CPI             2505
Unemployment     404
IsHoliday          2
dtype: int64

--------------------------------------------------

Sales DataFrame Information:
Store               45
Dept                81
Date               143
Weekly_Sales    359464
IsHoliday            2
dtype: int64

--------------------------------------------------

Store DataFrame Information:
Store    45
Type      3
Size     40
dtype: int64


# **Exploratory Data Analysis**

In [8]:
# change date format
feature_df['Date'] = pd.to_datetime(feature_df['Date'], format = 'mixed')
sales_df['Date'] = pd.to_datetime(sales_df['Date'], format = 'mixed')

In [9]:
# Merge different datasets
df = pd.merge(sales_df, feature_df, how = 'left', on = ['Store','Date','IsHoliday']) # merging sales and features by left since there are more rows for sales
df_new = pd.merge(df, store_df, how = 'left', on = 'Store') #merge df and df_stores by left to keep all the columns and rows

# Filling all NA values with 0
df_new = df_new.fillna(0)

In [10]:
df_new.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-05-02,24924.50,False,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,A,151315
1,1,1,2010-12-02,46039.49,True,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,A,151315
4,1,1,2010-05-03,21827.90,False,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,A,151315
5,1,1,2010-12-03,21043.39,False,57.79,2.667,0.0,0.0,0.0,0.0,0.0,211.380643,8.106,A,151315
6,1,1,2010-03-19,22136.64,False,54.58,2.720,0.0,0.0,0.0,0.0,0.0,211.215635,8.106,A,151315
7,1,1,2010-03-26,26229.21,False,51.45,2.732,0.0,0.0,0.0,0.0,0.0,211.018042,8.106,A,151315
8,1,1,2010-02-04,57258.43,False,62.27,2.719,0.0,0.0,0.0,0.0,0.0,210.820450,7.808,A,151315
9,1,1,2010-09-04,42960.91,False,65.86,2.770,0.0,0.0,0.0,0.0,0.0,210.622857,7.808,A,151315


In [11]:
# Dataset information
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Temperature   421570 non-null  float64       
 6   Fuel_Price    421570 non-null  float64       
 7   MarkDown1     421570 non-null  float64       
 8   MarkDown2     421570 non-null  float64       
 9   MarkDown3     421570 non-null  float64       
 10  MarkDown4     421570 non-null  float64       
 11  MarkDown5     421570 non-null  float64       
 12  CPI           421570 non-null  float64       
 13  Unemployment  421570 non-null  float64       
 14  Type          421570 non-null  object        
 15  Size          421

In [12]:
# Checking the number of unique values for each column in the dataset
df_new.nunique()

,0
Store,45
Dept,81
Date,143
Weekly_Sales,359464
IsHoliday,2
Temperature,3528
Fuel_Price,892
MarkDown1,2278
MarkDown2,1499
MarkDown3,1662


In [13]:
#seperate Numerical and Categorical Variables
cat_cols =['Store', 'Dept', 'IsHoliday', 'Type']
num_cols = df_new.columns.drop(cat_cols)
num_cols = num_cols.drop('Date') # Drop 'Date' column from numerical columns

print('Categorical Variables:')
print(cat_cols)

print('Numerical Variables:')
print(num_cols)

Categorical Variables:
['Store', 'Dept', 'IsHoliday', 'Type']
Numerical Variables:
Index(['Weekly_Sales', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size'],
      dtype='object')
